## 3조 프로젝트
### 주제: 데이터로 추적하는 넷플릭스 구독자들의 이탈 신호와 유지 전략

In [ ]:
### **화면 1: 넷플릭스 구독자 현황**
- **상단 Metric:** 전체 활성 사용자 수(DAU), 평균 구독 기간(LTV), 이번 달 이탈률(Churn Rate).
- **멤버십별 비교:** 베이식 vs 프리미엄 등 요금제별 이탈률 차이 시각화.
- **인사이트:** "광고형 요금제 가입자가 일반 요금제보다 이탈률이 높은가?"에 대한 답을 제공합니다.

In [ ]:
%%writefile module/project.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# ======================================================== 1.페이지 설정 =============================================================

st.set_page_config(page_title="넷플릭스 대시보드", layout="wide", initial_sidebar_state="expanded")

# ======================================================== 2. 데이터 =================================================================
@st.cache_data 
def load_data():
    data = {
        'Month': pd.date_range(start='2025-01-01', periods=12, freq='MS'),
        'Subscribers': [1250, 1270, 1321, 1345, 1360, 1393, 1410, 1425, 1440, 1465, 1490, 1516],
        'Retention': [12.1, 12.5, 11.8, 13.0, 14.2, 13.5, 12.8, 13.2, 14.5, 15.0, 14.8, 15.2],
    }
    df = pd.DataFrame(data)

    df['Prev_Subscribers'] = df['Subscribers'].shift(1)
    df['Growth_Rate'] = ((df['Subscribers'] - df['Prev_Subscribers']) / df['Prev_Subscribers']) * 100

    age_df = pd.DataFrame({
        '연령대': ['10대 이하', '20대', '30대', '40대', '50대', '60대 이상'],
        '비중': [8, 22, 25, 24, 15, 6]
    })

    gender_df = pd.DataFrame({
        '성별': ['여성', '남성'],
        '비중': [55, 45]
    })

    return {
        'main_df': df,
        'age_df': age_df,
        'gender_df': gender_df
    }

data_dict = load_data()
in_df = data_dict['main_df']
age_df = data_dict['age_df']
gender_df = data_dict['gender_df']

# ======================================================== 3.사이드바 구성=============================================================
with st.sidebar:
    st.header("🔍 분석 설정")
    month_labels = [d.strftime('%Y년 %m월') for d in in_df['Month']]
    selected_month = st.selectbox("분석 월 선택", options=month_labels, index=0)
    
    # 요금제 필터 제거됨
    analysis = st.button("🚀 데이터 분석 실행", use_container_width=True)
    st.divider()
    # 안내 문구에서 요금제 정보 제거
    st.info(f"💡 [현재 설정] 기간: **{selected_month}**")

# ======================================================== 4. 메인화면 구성=============================================================

header_col1, header_col2 = st.columns([1.5, 6])
with header_col1:
    st.image('https://upload.wikimedia.org/wikipedia/commons/0/08/Netflix_2015_logo.svg', width=250)
    
with header_col2:
    st.title('넷플릭스 구독자 현황 분석')
    st.text('💡 데이터로 추적하는 넷플릭스 구독자들의 이탈 신호와 유지 전략')
    
st.divider()

# ======================================================== 5. 유저 프로필 ====================================================

st.subheader("📊 넷플릭스 독점 유저 프로필 (2025년 8월 기준)")
col_left, col_right = st.columns(2)

with col_left:
    fig_age = px.bar(
        age_df, x='비중', y='연령대', 
        orientation='h', text_auto='%',
        title="📅 연령분포: 3040 세대가 주력",
        color_discrete_sequence=['#F6B6B6']
    )
    st.plotly_chart(fig_age, use_container_width=True)

with col_right:
    fig_gender = px.pie(
        gender_df, values='비중', names='성별',
        hole=0.5, title="👫 성별비중: 여성 유저 과반수",
        color_discrete_sequence=['#2E4766', '#8C9DB1']
    )
    fig_gender.update_traces(textinfo='percent+label')
    st.plotly_chart(fig_gender, use_container_width=True)

st.divider()

# ======================================================== 6. 분석 로직 ==================================================

if analysis:
    month_num = int(selected_month.split(' ')[1].replace('월', ''))
    target_df = in_df.iloc[month_num - 1:month_num] 
    
    if not target_df.empty:
        latest_data = target_df.iloc[0]
        latest_mau = latest_data['Subscribers'] / 100  
        growth_rate = latest_data['Growth_Rate']


        st.subheader(f"📊 {selected_month} 분석 결과")

        col3, col4 = st.columns(2)
        with col3:
            delta_text = f"{growth_rate:.2f}% (전월 대비)" if pd.notnull(growth_rate) else "신규 데이터"
            st.metric(
                label="📈 월 가입자 수", 
                value=f"{latest_mau:.2f} M", 
                delta=delta_text
            )
            st.caption("(가입자 수): 전체 체급 지표")
            
        with col4:
            st.metric(label="⏳ 유지 기간", value=f"{latest_data['Retention']}개월", delta="0.5개월")
            st.caption("(유지 기간): 수익성 지표")
        st.divider()

In [ ]:
    # 분석 2: 요금제별 이탈률 (Churn Rate)
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
def set_korean_font():
    if os.name == 'nt':
        plt.rc('font', family='Malgun Gothic')
    plt.rc('axes', unicode_minus=False)

set_korean_font()

st.set_page_config(layout="wide", page_title="이탈 예측 및 대응 전략", page_icon="📉")
st.title("📉 이탈 예측 모델링 및 골든타임 도출")
st.markdown("### : 데이터가 말해주는 '언제', '누구를', '어떻게' 잡아야 하는가")
st.markdown("---")

st.header("1. 이탈 골든타임")
st.info("💡 이탈 확정 유저들의 4주간 행동 패턴 추적 결과")

weeks = ['4주 전', '3주 전', '2주 전', '1주 전']
frequency = [5.2, 4.1, 2.3, 0.8]
completion = [75, 60, 45, 20]

col1, col2 = st.columns([2, 1])

with col1:
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    bars = ax1.bar(weeks, frequency, color='#dddddd', label='주간 접속 횟수', alpha=0.7, width=0.5)
    ax1.set_ylabel("주간 접속 횟수 (회)", fontsize=12)
    ax1.set_ylim(0, 6)
    
    ax2 = ax1.twinx()
    line = ax2.plot(weeks, completion, color='#E50914', marker='o', linewidth=3, markersize=10, label='콘텐츠 완독률')
    ax2.set_ylabel("완독률 (%)", fontsize=12, color='#E50914')
    ax2.tick_params(axis='y', labelcolor='#E50914')
    ax2.set_ylim(0, 100)

    ax2.annotate('Warning\n(Golden Time)', 
                 xy=(2, 45), xytext=(2, 65),
                 arrowprops=dict(facecolor='black', shrink=0.05),
                 ha='center', fontsize=12, fontweight='bold', color='red')

    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper left')
    ax1.set_title("이탈 D-4주 행동 변화 추이", fontsize=15)
    st.pyplot(fig1)

with col2:
    st.markdown("""
    **[데이터 포인트]**
    * **4주 전**: 접속 5.2회, 완독률 75% (정상)
    * **2주 전**: 접속 2.3회, 완독률 45% (**급감**)
    * **결론**: 접속 횟수가 반토막 나고, 완독률이 50% 밑으로 떨어지는 **'2주 전'**이 마케팅이 개입해야 할 유일한 골든타임입니다.
    """)

st.markdown("---")

st.header("2. 위험군 식별: \"14일의 법칙 (Red-line)\"")
st.info("💡 마지막 접속일(Recency) 경과에 따른 이탈 확률 상관관계")

np.random.seed(42)
recency_days = np.random.randint(1, 31, 200)
def churn_prob(day):
    base_prob = 1 / (1 + np.exp(-(day - 8) * 0.4)) 
    noise = np.random.normal(0, 0.05)
    prob = base_prob + noise
    return np.clip(prob * 100, 0, 100)

churn_probs = [churn_prob(d) for d in recency_days]
df_scatter = pd.DataFrame({'Recency': recency_days, 'ChurnProb': churn_probs})

col3, col4 = st.columns([2, 1])

with col3:
    fig2, ax3 = plt.subplots(figsize=(10, 6))
    colors = ['red' if x >= 14 else 'blue' for x in df_scatter['Recency']]
    ax3.scatter(df_scatter['Recency'], df_scatter['ChurnProb'], c=colors, alpha=0.6, edgecolors='w', s=80)
    ax3.axvline(x=14, color='red', linestyle='--', linewidth=2)
    ax3.text(14.5, 10, '이탈 레드라인\n(14일)', color='red', fontsize=12, fontweight='bold')
    ax3.annotate('7일 경과\n(이탈확률 45%)', xy=(7, 45), xytext=(2, 60),
                 arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=10)
    ax3.annotate('14일 경과\n(이탈확률 82%)', xy=(14, 82), xytext=(16, 90),
                 arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=10, fontweight='bold', color='red')
    ax3.set_title("마지막 접속 경과일(Recency) vs 이탈 확률", fontsize=15)
    ax3.set_xlabel("마지막 접속 후 경과일 (Day)")
    ax3.set_ylabel("이탈 확률 (%)")
    ax3.set_xlim(0, 31)
    ax3.set_ylim(0, 105)
    ax3.grid(True, linestyle='--', alpha=0.5)
    st.pyplot(fig2)

with col4:
    st.markdown("""
    **[Red-Line 분석]**
    * **7일 차**: 이탈 확률 45% (주의 단계)
    * **14일 차**: 이탈 확률 **82%** (복구 불가능)
    * **전략**: 사용자가 **7일~14일 사이** 구간에 진입했을 때, 강력한 푸시 알림과 복귀 혜택을 쏴야 합니다. 14일이 지나면 돌아오지 않습니다.
    """)

st.markdown("---")

st.header("3. 현재 구독자 진단: \"우리는 누구에게 집중해야 하는가\"")
st.info("💡 행동 데이터를 기반으로 분류한 전체 구독자 현황")

labels = ['안정군 (Active)', '주의군 (At-risk)', '위험군 (Churn-imminent)']
sizes = [70, 20, 10]
colors = ['#4CAF50', '#FF9800', '#F44336']
explode = (0, 0, 0.1)

col5, col6 = st.columns([1, 1])

with col5:
    fig3, ax4 = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax4.pie(sizes, explode=explode, labels=labels, colors=colors,
                                       autopct='%1.1f%%', shadow=True, startangle=140,
                                       textprops={'fontsize': 12})
    plt.setp(autotexts, size=14, weight="bold", color="white")
    ax4.set_title("전체 구독자 리스크 등급 분포", fontsize=15)
    st.pyplot(fig3)

with col6:
    st.markdown("#### 📋 그룹별 정의 및 Action Plan")
    st.success("**🟢 안정군 (Active) - 70%**\n* 주 3회 이상 접속, 완독률 70% 이상\n* **Action**: 건드리지 않음 (Natural Retention)")
    st.warning("**🟠 주의군 (At-risk) - 20%**\n* 접속 주기 불규칙, 검색만 하고 시청 안 함\n* **Action**: '찜한 콘텐츠' 알림, 인기작 추천")
    st.error("**🔴 위험군 (Churn-imminent) - 10%**\n* **7일 이상 미접속**, 3개월 차 진입\n* **Action**: **즉시 개입!** (특별 할인 쿠폰, 1:1 메시지)")